In [ ]:
import pygtrie
fdict = open("dict.opcorpora.txt", 'r')

resdict = {}
num_by_word = {}
cur = -1

suff = pygtrie.CharTrie()

def add_inv(word, i):
    for i in range(len(word) + 1):
        if word[:i] not in suff:
            suff[word[:i]] = set()
        suff[word[:i]].add(i)

for line in fdict:
    if line.strip() == "":
        continue
    words = line.split('\t')
    if (len(words)) == 1:
        cur = int(words[0].strip())
        continue
    if (len(words)) == 0:
        continue
    word, info = words
    if "Abbr" in info.split(',')[0].strip():
        continue
    if cur not in resdict:
        resdict[cur] = (info.split(',')[0].strip(), [])
    word = word.lower().replace('ё', 'е')
    invword = word[-1::-1]
    add_inv(invword, cur)
    resdict[cur][1].append(word)
    if word not in num_by_word:
        num_by_word[word] = set()
    num_by_word[word].add(cur)

In [2]:
f = open("dict.opcorpora.xml", "r")

types = []
links = []
for line in f:
    if line.strip().startswith("<type"):
        types.append(line)
    if line.strip().startswith("<link"):
        links.append(line)


In [3]:
types

['<type id="1">ADJF-ADJS</type>\n',
 '<type id="2">ADJF-COMP</type>\n',
 '<type id="3">INFN-VERB</type>\n',
 '<type id="4">INFN-PRTF</type>\n',
 '<type id="5">INFN-GRND</type>\n',
 '<type id="6">PRTF-PRTS</type>\n',
 '<type id="7">NAME-PATR</type>\n',
 '<type id="8">PATR_MASC-PATR_FEMN</type>\n',
 '<type id="9">SURN_MASC-SURN_FEMN</type>\n',
 '<type id="10">SURN_MASC-SURN_PLUR</type>\n',
 '<type id="11">PERF-IMPF</type>\n',
 '<type id="12">ADJF-SUPR_ejsh</type>\n',
 '<type id="13">PATR_MASC_FORM-PATR_MASC_INFR</type>\n',
 '<type id="14">PATR_FEMN_FORM-PATR_FEMN_INFR</type>\n',
 '<type id="15">ADJF_eish-SUPR_nai_eish</type>\n',
 '<type id="16">ADJF-SUPR_ajsh</type>\n',
 '<type id="17">ADJF_aish-SUPR_nai_aish</type>\n',
 '<type id="18">ADJF-SUPR_suppl</type>\n',
 '<type id="19">ADJF-SUPR_nai</type>\n',
 '<type id="20">ADJF-SUPR_slng</type>\n',
 '<type id="21">FULL-CONTRACTED</type>\n',
 '<type id="22">NORM-ORPHOVAR</type>\n',
 '<type id="23">CARDINAL-ORDINAL</type>\n',
 '<type id="24">SB

In [4]:
links[:10]

['<link_types>\n',
 '<links>\n',
 '    <link id="1" from="5" to="6" type="1"/>\n',
 '    <link id="2" from="5" to="7" type="2"/>\n',
 '    <link id="3" from="9" to="8" type="3"/>\n',
 '    <link id="4" from="9" to="10" type="4"/>\n',
 '    <link id="5" from="9" to="12" type="5"/>\n',
 '    <link id="6" from="10" to="11" type="6"/>\n',
 '    <link id="7" from="14" to="13" type="3"/>\n',
 '    <link id="8" from="14" to="15" type="4"/>\n']

In [5]:
links = links[2:]

In [6]:
go_to = {}
for link in links:
    fr, to = link.strip().split()[2:4]
    fr = int(fr[6:-1])
    to = int(to[4:-1])
    if to not in go_to:
        go_to[to] = []
    go_to[to].append(fr)

In [7]:
go_to[10]


[9]

In [24]:
import random

def get_all_reachables(num, was):
    if num in was:
        return was
    was.add(num)
    if num not in go_to:
        return was
    for i in go_to[num]:
        found = get_all_reachables(i, was)
        was = was.union(found)
    return was
def get_with(l, w):
    all = []
    for p in l:
        if resdict[p][0].startswith(w):
            all.append(p)
    return -1 if all == [] else random.choice(all)

In [28]:
def find_right_num(word):
    ans = set()
    for num in num_by_word[word]:
        ans = ans.union(get_all_reachables(num, set()))
    ans = list(ans)
    a = get_with(ans, "ADJF")
    if a != -1:
        return a
    a = get_with(ans, "NOUN")
    if a != -1:
        return a
    a = get_with(ans, "INFN")
    if a != -1:
        return a
    a = get_with(ans, "CONJ")
    if a != -1:
        return a
    a = get_with(ans, "INTJ")
    if a != -1:
        return a
    a = get_with(ans, "PREP")
    if a != -1:
        return a
    return ans[0]

In [32]:
def get_naming(w):
    if w.startswith("NOUN"):
        return "S"
    if w.startswith("INFN"):
        return "V"
    if w.startswith("ADJF"):
        return "A"
    if w.startswith("PREP"):
        return "PR"
    if w.startswith("CONJ"):
        return "CONJ"
    return "ADV"
    
pop_adv = ['уже', 'же', 'еще']
pop_conj = ["и", "а", "но", "да", "если", "что", "когда"]
pop_pr = ["за", "для", "в", "о", "к", "из", "от", "по", "под", "с", "об", "обо", "до", "над", "на", "ко", "к", "без", "из", "у"]

def make_sentence(sent):
    res = []
    for word in sent:
        loword = word.lower().replace('ё', 'е')
        if loword in pop_adv:
            res.append(word + "{" + loword + "=ADV}")
        elif loword in pop_conj:
            res.append(word + "{" + loword + "=CONJ}")
        elif loword in pop_pr:
            res.append(word + "{" + loword + "=PR}")
        elif loword in num_by_word:
            num = find_right_num(loword)
            res.append(word + "{"+resdict[num][1][0] + "=" + get_naming(resdict[num][0]) + "}")
        else:
            res.append(word + "{"+word.lower() + "=NI}")
    return res

In [33]:
fin = open("test.txt", 'r')
fout = open("output.txt", 'w')
for line in fin:
    words = line.replace('.', '').replace(',', '').replace('?', '').replace('!', '').strip().split()
    print(" ".join(make_sentence(words)), file=fout)
fout.close()

In [123]:
num_by_word["к"]

{127460, 389619, 390191, 390192}

In [124]:
print(resdict[127460], resdict[128190], resdict[389619], resdict[390191], resdict[390192])

('PREP', ['к', 'ко']) ('CONJ', ['как', 'како']) ('NOUN', ['к', 'к', 'к', 'к', 'к', 'к', 'к', 'к', 'к', 'к', 'к', 'к']) ('NOUN', ['к', 'к', 'к', 'к', 'к', 'к']) ('NOUN', ['к', 'к', 'к', 'к', 'к', 'к'])


In [127]:
print(make_sentence(["Эти", "типы", "стали", "есть", "на", "складе"]))

['Эти{этот=A}', 'типы{тип=S}', 'стали{стать=V}', 'есть{есть=ADV}', 'на{на=ADV}', 'складе{склад=S}']


In [117]:
num_by_word["есть"]

{37265, 95396, 95398}

In [119]:
print(resdict[37265], resdict[95396], resdict[95398])

('VERB', ['есть', 'суть', 'есть', 'был', 'была', 'было', 'были', 'буду', 'будем', 'будешь', 'будете', 'будет', 'будут', 'будем', 'будь', 'будьте', 'е', 'есмь']) ('INTJ', ['есть']) ('INFN', ['есть'])


In [22]:
num_by_word['стадий']

{334408, 334417}

In [23]:
resdict[334408]

('NOUN',
 ['стадий',
  'стадия',
  'стадию',
  'стадий',
  'стадием',
  'стадии',
  'стадии',
  'стадиев',
  'стадиям',
  'стадии',
  'стадиями',
  'стадиях'])